# Функторы и монады  
<br>
![](http://imgs.xkcd.com/comics/hypotheticals.png)
<br>

### Функторы и с чем их едят

## Что такое ковариантный функтор

>С точки зрения «синтаксического подхода», ковариантным функтором является всякий тип (назовем его X) имеющий type parameter (назовем его T) с методом, который имеет следующую сигнатуру (назовем его map)

```
trait X[T] {
  def map(f: T => R): X[R]
}
```
<br>
«Синтаксический подход» хорош тем, что он позволяет свести в общую схему многие категориальные конструкции
<br>
```
trait X[T] {
  // ковариант
  def map[R](f: T => R): X[R]

  // контрвариант
  def contramap[R](f: R => T): X[R]

  // инвариант
  def xmap[R](f: (T => R, R => T)): X[R]

  // апликативный функтор
  def apply[R](f: X[T => R]): X[R]

  // монада
  def flatMap[R](f: T => X[R]): X[R]

  // комонада
  def coflatMap[R](f: X[T] => R): X[R]
}
```


| функтор  | аргументы  |
|---|---|
|контрвариант| A => B  |
|инвариант|  B => A |
|инвариант| (A => B, B => A)  |
|апликативный|  F[A => B] |
|монада| A => F[B]  |
|комонада| F[A] => B  |

Функтор - это контейнер. Он может хранить либо ничего, либо само значение
<br><br>
![](https://hsto.org/storage2/be0/182/2de/be01822de6f660845c952b2b4fa7edb6.png)
<br>

Примеры ковариантных функторов из стандартной библиотеки scala

## Option

In [1]:
// Наш первый пациент
Option

res0: Option.type = scala.Option$@61a71e63

Одна из первых монад, которую знают многие программисты - это монада *ковариантный функтор* Option. Так же известная как Maybe монада в Haskell и Optional в Java 8. Вы можете думать об этой монаде, как о коллекции, которая состоит из одного элемента. Но при этом она может либо содержать его, либо не содержать. Таким образом вы можете оборачивать результат в эту монаду. Даже тот результат, который может зафейлиться. Если результат не зафейлится, он вернёт значение и это значение вернётся обёрнутой в монаду. В противном случае там будет лежать None. Мы можем изменять монаду Option с помощью функции map.

In [2]:
val three = Option(3)

three: Option[Int] = Some(3)

In [3]:
val twelve = three map (_ * 4)

twelve: Option[Int] = Some(12)

Но когда мы захотим положить несколько значений в эту монаду, у нас получится Option в Option

```
Option[Option[Int]]
```

In [4]:
val four = Option(4)

four: Option[Int] = Some(4)

In [5]:
val twelveB = three map (i => four map (i * _))

twelveB: Option[Option[Int]] = Some(Some(12))

^ тут мы получили Option в Option, так сказать, монада на монаде. Конечно, это не то, что мы хотим, поэтому давайте попробуем использовать flatMap

![](https://hsto.org/storage2/983/73b/546/98373b54695813ce070a7fc782ca8a35.png)

In [6]:
val twelveC = three flatMap (i => four map (i * _))

twelveC: Option[Int] = Some(12)

flatMap — парень с улицы, flatMap знает толк в контекстах. Уж он-то в курсе, как применить функцию к упакованному в контекст значению.

In [7]:
val twelveD = for {
  i <- three
  j <- four
} yield (i * j)

twelveD: Option[Int] = Some(12)

Не забудем о том, что монада заботится о том, что наше значение может быть и пустым

In [8]:
val oops: Option[Int] = None

oops: Option[Int] = None

![](https://hsto.org/storage2/788/3ad/937/7883ad93713ea4406c129d54f83659d8.png)

In [9]:
val oopsB = for {
  i <- three
  j <- oops
} yield (i * j)

oopsB: Option[Int] = None

In [10]:
val oopsC = for {
  i <- oops
  j <- four
} yield (i * j)

oopsC: Option[Int] = None

![](https://hsto.org/storage2/238/88c/8ed/23888c8ed4ae153e5c4d62321fec63c1.png)
<p style="font-size: 10px">Билл О'Рейли ничегошеньки не смыслит в функторе Maybe </p>

H1: Option — «как бы контейнер» на один элемент, где может что-то лежит (Some), а может и нет (None)

## List

In [11]:
// Наш второй пациент
List

res10: List.type = scala.collection.immutable.List$@5e63475

А вот ещё один пример: что происходит, когда вы применяете функцию к списку?

![](https://hsto.org/storage2/5ba/e29/3d1/5bae293d123881aa5920d3e7e9e1d039.png)

Списки тоже функторы!

H2: List — контейнер, в котором может быть 0...N элементов

## Future

In [12]:
import scala.concurrent.duration._
import scala.concurrent.{Future,ExecutionContext,Await}
import ExecutionContext.Implicits.global

import scala.concurrent.duration._

import scala.concurrent.{Future,ExecutionContext,Await}

import ExecutionContext.Implicits.global

In [13]:
// Третий на подходе
Future

res12: Future.type = scala.concurrent.Future$@ad4eb67

Существует монада для параллельного вычесления и асинхронности. Future монада существует для обёртки медленных вычислений, для того чтобы засунуть их в другой поток. Если вы зовёте Future монаду - она возращается тут же, позволяя вызывать поток, пока дополнительные потоки делают вычисления. Т.е. когда мы вызываем Future - она ещё не содержит значение, но через какое-то время оно (возможно) там появится.

Создадим 2 монады Future, каждая из которых будет спать 10 секунд. В главном потоке просуммируем f1 И f2. 

In [14]:
val f1=Future{
  Thread.sleep(10000)
  1 }

f1: Future[Int] = Future(<not completed>)

In [15]:
val f2=Future{
  Thread.sleep(10000)
  2 }

f2: Future[Int] = Future(<not completed>)

Заметим, что f1 и f2 выполняться вместе за 10 секунд в разных потоках.

In [16]:
val f3=for {
  v1 <- f1
  v2 <- f2
  } yield (v1+v2)

f3: Future[Int] = Future(<not completed>)

In [17]:
println(Await.result(f3, 30.seconds))

3


H3: Future — «как бы контейнер» на один элемент, где может что-то уже лежит, может будет лежать, или уже лежит исключение, или будет лежать исключение или никогда ничего лежать не будет.

## Try

In [20]:
import scala.util.Try

import scala.util.Try

In [22]:
// Ох, сколько больных развелось
Try

res21: Try.type = scala.util.Try$@120f1e89

Try монада существует для того, чтобы не оборачивать ошибки с помощью Either. Об Either поговорим чуть попозже, потому что это BiFunctor. Основная идея в том, что у вас лежит либо элемент, либо ошибка.

In [27]:
// Создадим функцию, которая будет возвращать монаду
def f(x: Int, y: Int): Try[Int] = Try(x / y)

defined function f

In [28]:
f(1, 1)

res27: Try[Int] = Success(1)

In [29]:
f(1, 0)

res28: Try[Int] = Failure(java.lang.ArithmeticException: / by zero)

Монада, как и было сказанно выше принимает 2 значения. Либо ```Success``` либо ```Failure```

In [32]:
// Узнаем, всё ли хорошо
f(1, 1).isSuccess

res31: Boolean = true

Литература для создания тетрадки

1. [Functors and things using Scala](http://blog.tmorris.net/posts/functors-and-things-using-scala/index.html)
2. [Функторы, аппликативные функторы и монады в картинках](https://habrahabr.ru/post/183150/)
3. [First steps with monads in Scala](https://darrenjw.wordpress.com/2016/04/15/first-steps-with-monads-in-scala/)
4. [FP на Scala: Что такое функтор?](https://habrahabr.ru/company/golovachcourses/blog/266905/)
5. [Scala's Either, Try and the M word](https://mauricio.github.io/2014/02/17/scala-either-try-and-the-m-word.html)


<p style='font-size: 10px'>[Если вы классный](http://homepages.inf.ed.ac.uk/wadler/papers/marktoberdorf/baastad.pdf)</p>

<p style='font-size: 10px'>[Очень-очень классный](https://bartoszmilewski.com/2014/10/28/category-theory-for-programmers-the-preface/)</p>
